# Importing libraries

In [1]:
import pandas as pd
import io
import os

# Concatenating raw jsons

In [37]:
#reading raw json
raw_json_file_names = []
raw_jsons = []

for file in os.listdir("."):
    if file.startswith("run_results_"):
        raw_json_file_names.append(file)
        
for file in raw_json_file_names:
    with open(file,'r') as filestream:
        raw_jsons.append(pd.read_json(filestream))

In [38]:
#removing unnessasary stuff
for json_table in raw_jsons:
    del json_table['remove']
    del json_table['listingValue']
    json_table.columns = ['html']
    json_table['html_str'] = json_table['html'].astype(str)
    json_table = json_table.drop_duplicates(subset='html_str')

In [39]:
#concatenating jsons and removing duplicates
json_all = raw_jsons[0]
for json_table in raw_jsons[1:]:
    json_all = pd.concat([json_all,json_table], axis=0, ignore_index=True)
json_all = json_all.drop_duplicates(subset="html_str")
del json_all["html_str"]
json_all.reset_index(drop=True, inplace=True)

In [ ]:
#output raw json without duplicate as dataframe in pickle
with open('raw_json_no_dups.json','w') as fout:
    json_all.to_json(fout,orient='table')

# Getting only user messages
Also parsing links to message and authors

In [54]:
from bs4 import BeautifulSoup

In [55]:
#to add empty string in case of empty search
link_search = lambda s: '' if s is None else s['href']

In [56]:
#scrap user messages 
def scrapFromHTML(soup, row):
    row['author'] = "https://opendatascience.slack.com" + link_search(soup.find("a", class_="c-link c-message__sender_link"))
    row['link'] = link_search(soup.find("a", class_="c-link c-timestamp c-timestamp--static"))
    tag = soup.find("span", class_="c-message__body")
    if not tag is None:
        row['message'] = tag.get_text("\n")

In [57]:
#load raw table
with open('raw_json_no_dups.json','r') as fin:
    html_table = pd.read_json(fin,compression=None,orient='table')

In [58]:
#add tables for future scrapped data
html_table['author'] = ""
html_table['link'] = ""
html_table['message'] = ""

In [59]:
#scrapping of new information
for i in html_table.index:
    html_doc = html_table['html'][i]['name']
    soup = BeautifulSoup(html_doc,'html5lib')
    scrapFromHTML(soup, html_table.loc[i,:])

In [60]:
#copy table without non-user messages
html_table_usr_msg = html_table[html_table.author!="https://opendatascience.slack.com"]
html_table_usr_msg.reset_index(drop=True, inplace=True)
del html_table_usr_msg['html']

In [61]:
#output user messages table
with open("user_messages.json",'w') as fout:
    html_table_usr_msg.to_json(fout, compression=None, orient='table')

# Getting numbers out of text
And removing messages without numbers

In [2]:
import re

In [65]:
#input user messages table
with open("user_messages.json",'r') as fin:
    usr_msg = pd.read_json(fin, compression=None, orient='table')

In [66]:
usr_msg['numbers'] = usr_msg.apply(lambda x: [], axis=1)
usr_msg['hasNumbers'] = False

In [67]:
#scrapping numbers in messages
for i in usr_msg.index:
    msg = usr_msg['message'][i]
    usr_msg['numbers'][i] = re.findall(r'[0-9]+', msg)
    if usr_msg['numbers'][i]:
        usr_msg['hasNumbers'][i] = True 

/usr/lib/python3/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [68]:
#copy table without only-word messages (other are candidates for having vacancy information)
usr_msg_clear = usr_msg[usr_msg.hasNumbers]
usr_msg_clear.reset_index(drop=True, inplace=True)
del usr_msg_clear['hasNumbers']

In [69]:
#output clear user messages table
with open("user_messages_clear.json",'w') as fout:
    usr_msg_clear.to_json(fout, compression=None, orient='table')

# Parsing data and time

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from tqdm import tqdm
import threading

In [35]:
def scrapDateTime(driver, row, link):

    driver.get(link.replace('archives','messages'))
#    elem.send_keys("Posted using Python's Selenium WebDriver bindings!")
#    elem = driver.find_element_by_id("signin_btn")
#    elem.click()
    try:
        WebDriverWait(driver,15).until(
             EC.presence_of_element_located((By.CLASS_NAME,'p-archives_banner')))
    except TimeoutException:
        pass
    
    try:
        WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.CLASS_NAME,'c-message_list__day_divider__label__pill')))
        elem1 = driver.find_element_by_class_name('c-message_list__day_divider__label__pill')
        row['date'] = elem1.text
    except TimeoutException:
        pass
    except StaleElementReferenceException:
        elem1 = driver.find_element_by_class_name('c-message_list__day_divider__label__pill')
        row['date'] = elem1.text
        
    try:
        WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.CLASS_NAME,'c-message__content_header')))
        elems = driver.find_elements_by_class_name('c-message__content_header')
        for elem in elems:
            if link in elem.get_attribute('innerHTML'):
                row['time'] = re.search('data-stringify-suffix="]"(.+?)</span>', elem.get_attribute('innerHTML')).group(1)
    except TimeoutException:
        pass
    except StaleElementReferenceException:
        elems = driver.find_elements_by_class_name('c-message__content_header')
        for elem in elems:
            if link in elem.get_attribute('innerHTML'):
                row['time'] = re.search('data-stringify-suffix="]"(.+?)</span>', elem.get_attribute('innerHTML')).group(1)

In [5]:
#input user messages table
with open("user_messages_clear.json",'r') as fin:
    usr_msg_clear = pd.read_json(fin, compression=None, orient='table')

In [6]:
usr_msg_clear['date'] = ''
usr_msg_clear['time'] = ''

In [45]:
usr = ""
pwd = ""

batch_size = 3

options = webdriver.chrome.options.Options()
options.headless = True

drivers = []

for i in range(batch_size):
    driver = webdriver.Chrome('/opt/chromedriver', options=options)
    driver.get('https://opendatascience.slack.com/')

    elem = driver.find_element_by_id("email")
    elem.send_keys(usr)
    elem = driver.find_element_by_id("password")
    elem.send_keys(pwd)
    elem.send_keys(Keys.RETURN)
    
    drivers.append(driver)

empty_rows = usr_msg_clear.index[usr_msg_clear['date'] == '']

for i in tqdm(empty_rows[::batch_size]):
    threads = []
    for j in range(min(batch_size,empty_rows[-1]-i)):
        k = empty_rows[empty_rows.get_loc(i)+j]
        thread = threading.Thread(target=scrapDateTime, args=[drivers[j], usr_msg_clear.loc[k,:], usr_msg_clear['link'][k]])
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()

for driver in drivers:
    driver.quit()








  0%|          | 0/186 [00:00<?, ?it/s]






  1%|          | 1/186 [00:20<1:01:41, 20.01s/it]






  1%|          | 2/186 [00:39<1:00:50, 19.84s/it]






  2%|▏         | 3/186 [00:55<57:09, 18.74s/it]  






  2%|▏         | 4/186 [01:11<54:05, 17.83s/it]






  3%|▎         | 5/186 [01:26<51:22, 17.03s/it]






  3%|▎         | 6/186 [01:42<50:23, 16.80s/it]






  4%|▍         | 7/186 [01:58<49:22, 16.55s/it]






  4%|▍         | 8/186 [02:14<48:48, 16.45s/it]






  5%|▍         | 9/186 [02:31<48:39, 16.49s/it]






  5%|▌         | 10/186 [02:47<48:14, 16.45s/it]






  6%|▌         | 11/186 [03:03<47:22, 16.24s/it]






  6%|▋         | 12/186 [03:19<47:01, 16.22s/it]






  7%|▋         | 13/186 [03:36<46:50, 16.25s/it]






  8%|▊         | 14/186 [03:52<46:18, 16.15s/it]






  8%|▊         | 15/186 [04:07<45:11, 15.86s/it]






  9%|▊         | 16/186 [04:22<44:43, 15.78s/it]






  9%|▉         | 17/186 [04:39<44:53, 15.94s/it]






 10%|▉         

 58%|█████▊    | 107/186 [28:30<20:49, 15.82s/it]






 58%|█████▊    | 108/186 [28:45<20:01, 15.40s/it]






 59%|█████▊    | 109/186 [28:59<19:29, 15.19s/it]






 59%|█████▉    | 110/186 [29:15<19:21, 15.28s/it]






 60%|█████▉    | 111/186 [29:29<18:46, 15.02s/it]






 60%|██████    | 112/186 [29:44<18:12, 14.77s/it]






 61%|██████    | 113/186 [29:59<18:21, 15.09s/it]






 61%|██████▏   | 114/186 [30:15<18:16, 15.22s/it]






 62%|██████▏   | 115/186 [30:31<18:13, 15.40s/it]






 62%|██████▏   | 116/186 [30:47<18:20, 15.72s/it]






 63%|██████▎   | 117/186 [31:03<18:08, 15.78s/it]






 63%|██████▎   | 118/186 [31:19<17:56, 15.84s/it]






 64%|██████▍   | 119/186 [31:36<18:10, 16.27s/it]






 65%|██████▍   | 120/186 [31:52<17:38, 16.04s/it]






 65%|██████▌   | 121/186 [32:09<17:35, 16.23s/it]






 66%|██████▌   | 122/186 [32:23<16:36, 15.58s/it]






 66%|██████▌   | 123/186 [32:42<17:40, 16.84s/it]






 67%|██████▋   | 124/186 [32:58<17:03, 16.51s/it

In [48]:
with open("user_messages_clear_w_dt.json",'w') as fout:
    usr_msg_clear.to_json(fout, compression=None, orient='table')

# Formatting date and time

In [103]:
import numpy as np
import time

def clean_up_time(x):
    return x.replace('>','')

def current_year_clean_up(x):
    if 'day' in x:
        x = x.replace(x[-2:],'')
        x = x.replace(x.split(', ')[0] + ', ','')
        return x + ' ' + str(time.localtime().tm_year)
    else:
        return x.replace(x.split(' ')[1][-3:],'')

In [106]:
with open("user_messages_clear_w_dt.json",'r') as fout:
    usr_msg_clear = pd.read_json(fout, compression=None, orient='table')

In [148]:
#DELETE ONE COMMENT AND EDIT ONE MESSAGE BASED ON THAT COMMENT
#usr_msg_clear.drop(usr_msg_clear[usr_msg_clear['date'].str.contains('Today')].index, axis=0, inplace=True)
index = usr_msg_clear[usr_msg_clear['link'] == 'https://opendatascience.slack.com/archives/C04DA5FUF/p1527178372000150'].index
usr_msg_clear.loc[index]['numbers'][1] = 80
usr_msg_clear.loc[index]['numbers'][2] = 180
usr_msg_clear.loc[index]['message'] = usr_msg_clear.loc[index]['message'].replace('50','80')
#print(usr_msg_clear.loc[usr_msg_clear['date'].str.contains('Today')])

In [149]:
usr_msg_clear['time'] = usr_msg_clear['time'].apply(np.vectorize(clean_up_time))
usr_msg_clear['date'] = usr_msg_clear['date'].apply(np.vectorize(current_year_clean_up))

In [154]:
usr_msg_clear['date'] = pd.to_datetime(usr_msg_clear['date'], format='%B %d %Y')
#usr_msg_clear['time'] = pd.to_datetime(usr_msg_clear['time'], format='%I:%M %p')
#have 65 examples without time, left unformatted for now

cols = ['date', 'time']

print(usr_msg_clear[cols])

           date      time
0    2019-05-29   2:06 PM
1    2019-05-29  12:46 PM
2    2019-05-29  12:50 PM
3    2019-05-29  12:31 PM
4    2019-05-29  12:32 PM
5    2019-05-28  10:38 AM
6    2019-05-28  11:46 AM
7    2019-05-24  12:24 AM
8    2019-05-25  11:24 AM
9    2019-05-26  12:42 PM
10   2019-05-27  11:33 AM
11   2019-05-27  12:46 PM
12   2019-05-27   2:39 PM
13   2019-05-27   4:28 PM
14   2019-05-28   4:40 PM
15   2019-05-28   5:07 PM
16   2019-05-20   5:09 PM
17   2019-05-20   6:02 PM
18   2019-05-21   5:45 PM
19   2019-05-21   7:43 PM
20   2019-05-22   5:30 PM
21   2019-05-23   6:12 PM
22   2019-05-23   7:49 PM
23   2019-05-23  12:52 PM
24   2019-03-04   3:07 PM
25   2019-03-04   3:07 PM
26   2019-03-04   3:07 PM
27   2019-05-06   1:19 PM
28   2019-03-04   3:07 PM
29   2019-05-09   8:00 PM
...         ...       ...
2402 2017-01-24   9:38 AM
2403 2017-01-13   5:17 PM
2404 2017-08-14   2:54 PM
2405 2017-08-14   4:07 PM
2406 2017-08-14   4:31 PM
2407 2017-08-14   8:42 PM
2408 2017-01

In [155]:
with open("user_messages_clear_w_dt.json",'w') as fout:
    usr_msg_clear.to_json(fout, compression=None, orient='table')